# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
% matplotlib inline

UsageError: Line magic function `%` not found.


In [2]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [3]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [4]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
portfolio.shape, profile.shape, transcript.shape

((10, 6), (17000, 5), (306534, 4))

## Data Exploration

Before performing any preprocessing for the above 3 datasets, we need to get a good understanding as to how the data looks like. Thus, I'm performing few data exploration steps.

### 1. Portfolio Data

In [7]:
# Check whether portfolio dataset contains any missing values
portfolio.isnull().sum(axis=0)

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64

In [8]:
# Available offer types
portfolio.offer_type.value_counts()

bogo             4
discount         4
informational    2
Name: offer_type, dtype: int64

In [9]:
# All available channels
avail_channels = []

for channels in portfolio['channels']:
    for channel in channels:
        if channel not in avail_channels:
            avail_channels.append(channel)

avail_channels

['email', 'mobile', 'social', 'web']

In [10]:
# check whether all offer data is unique
len(np.unique(portfolio['id'])) == portfolio.shape[0]

True

### 2. Profile Data

In [11]:
# Check whether profile dataset contains any missing values
profile.isnull().sum(axis=0)

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

In [12]:
#explore some statistics about the age column
profile['age'].describe()

count    17000.000000
mean        62.531412
std         26.738580
min         18.000000
25%         45.000000
50%         58.000000
75%         73.000000
max        118.000000
Name: age, dtype: float64

In [13]:
# Different values in gender column
profile['gender'].value_counts()

M    8484
F    6129
O     212
Name: gender, dtype: int64

In [14]:
# Checkout NaN values available in income column
profile[profile['income'].isnull()]

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN
...,...,...,...,...,...
16980,None,118,5c686d09ca4d475a8f750f2ba07e0440,20160901,NaN
16982,None,118,d9ca82f550ac4ee58b6299cf1e5c824a,20160415,NaN
16989,None,118,ca45ee1883624304bac1e4c8a114f045,20180305,NaN
16991,None,118,a9a20fa8b5504360beb4e7c8712f8306,20160116,NaN


In [15]:
profile[profile['income'].isnull()].age.value_counts()

118    2175
Name: age, dtype: int64

According to the above cell, all the profiles with NaNs for income are having 118 for age. Thus, it seems that these 2175 rows in profile dataset contains invalid data. And I prefer to clean up all these invalid data during the preprocessing step.

In [16]:
profile.dtypes

gender               object
age                   int64
id                   object
became_member_on      int64
income              float64
dtype: object

### 3. Transcript Data

In [17]:
# Check whether transcript dataset contains any missing values
transcript.isnull().sum(axis=0)

person    0
event     0
value     0
time      0
dtype: int64

In [18]:
# Different event types available
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

## Data Preprocessing

According to the previous Data Exploration step, it is clear that some data preprocessing has to be performed in all 3 datasets, in order to proceed with next steps in this task. Thus, below are the changes done to each of the datasets.

### 1. Portfolio Data

1. One hot encode channels
2. One hot encode offer_type
3. Convert duration (which is in days to hours)
4. Replace channels and offer-type columns with one hot encoded columns
5. Rename id column as offer_id

In [19]:
# one hot encode offer_type column
portfolio = pd.get_dummies(portfolio, columns=['offer_type'],prefix="", prefix_sep="")

In [20]:
# Add in the email, mobile, social, and web columns
for channel in avail_channels:
    portfolio[channel] = portfolio['channels'].apply(lambda x: 1 if channel in x else 0)  

# Drop the original channels column
portfolio.drop('channels', axis = 1, inplace=True) 

In [21]:
# create a new column duration_hours by converting duration column 
portfolio['duration_hours'] = portfolio['duration'].apply(lambda x: x * 24)

# Drop the original duration column
portfolio.drop('duration', axis = 1, inplace=True) 

In [22]:
# rename if column as offer_id
portfolio = portfolio.rename(columns={"id":"offer_id"})
portfolio.head()

,reward,difficulty,offer_id,bogo,discount,informational,email,mobile,social,web,duration_hours
0,10,10,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,1,1,0,168
1,10,10,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1,120
2,0,0,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0,1,96
3,5,5,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0,1,168
4,5,20,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0,1,240


### 2. Profile Data

1. Drop profiles with missing data
2. Convert became_member_on column to actual dates
3. Add a new column to identify the membership time period in days (to identify whether there's any relation)
4. Rename id column as customer_id

In [23]:
# before dropping the rows with missing data, we need to keep track of specific customer ids with missing data,
# so that we could remove those from transript data too
invalid_customer_ids = list(profile[profile['income'].isnull()]['id'])
len(invalid_customer_ids)

2175

In [24]:
# drop profiles with missing data
profile = profile.dropna(axis=0, subset=['income'])
profile.shape[0]   # 17000-2175=14825

14825

In [25]:
# convert became_member_on to actual date
profile['member_date'] = profile.became_member_on.apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))
profile.drop('became_member_on', axis = 1, inplace = True)

In [26]:
# add a new column membership_days to profile dataset
curr_datetime = datetime.datetime.now()
profile['membership_days']=profile['member_date'].apply(lambda x: (curr_datetime - x).days )

In [27]:
# Rename id column as customer_id
profile = profile.rename(columns={"id":"customer_id"})
profile.head()

,gender,age,customer_id,income,member_date,membership_days
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15,858
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017-05-09,925
5,M,68,e2127556f4f64592b11af22de27a7932,70000.0,2018-04-26,573
8,M,65,389bc3fa690240e798340f5a15918d5c,53000.0,2018-02-09,649
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017-11-11,739


### 3. Transcript Data

1. Rename person column to customer_id
2. Drop data which relates to invalid customer profiles
3. Convert value column into 2 columns; value and value_type (value column will be replaced with actual values)
4. One hot encode event column and rename the column names

In [28]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [29]:
# rename person column to customer_id
transcript = transcript.rename(columns={"person":"customer_id"})

In [30]:
# drop data which relates to invalid customer profiles
transcript = transcript[~transcript['customer_id'].isin(invalid_customer_ids)]

In [31]:
# Convert value column into 2 columns; value and value_type
transcript['value_type'] = transcript['value'].apply(lambda x: list(x.keys())[0])
transcript['value'] = transcript['value'].apply(lambda x: list(x.values())[0])

In [32]:
# one hot encode event column
transcript = pd.get_dummies(transcript, columns=['event'],prefix="", prefix_sep="")

In [33]:
# rename column names
transcript = transcript.rename(columns={"offer received":"offer_received", "offer viewed":"offer_viewed", "offer completed":"offer_completed"})

In [34]:
# check unique values in value_type column
transcript.value_type.value_counts()

amount      123957
offer id    116361
offer_id     32444
Name: value_type, dtype: int64

According to above cell, it seems that value_type column has 3 values, (amount, offer id and offer_id), when it should have only 2 different values amount and offer_id. Hence, we need to replace 'offer id' with 'offer_id'.

In [35]:
transcript['value_type'] = transcript['value_type'].apply(lambda x:'offer_id' if x=='offer id' else x)

In [36]:
# check unique values in value_type column after updating
transcript.value_type.value_counts()

offer_id    148805
amount      123957
Name: value_type, dtype: int64

### 4. Create customer_offer dataframe

In this step, my goal is to come up with a new dataframe, which includes data related to whether a customer completed a particular offer or not. This dataframe would contain these columns; customer_id, offer_id, completed.

The 'completed' column would contain binary values (1's and 0's)

Hence, as the first step for it, I need to filter out offer related rows from transcript dataframe. Afterwards, we can rename 'value' column as 'offer_id' and drop 'value_type' and 'transaction' columns, since they don't contain any important information for this task. 

In [37]:
# Filter out offer details from transcript dataframe
offer_data = transcript[transcript['value_type']=='offer_id']
offer_data.shape[0]

148805

In [38]:
# do necessary changes to offer_data 
offer_data = offer_data.rename(columns={'value':'offer_id'})
offer_data.drop(labels=['value_type','transaction'], axis = 1, inplace=True)
offer_data.head()

,customer_id,offer_id,time,offer_completed,offer_received,offer_viewed
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0,1,0
2,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,0,1,0
5,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,0,1,0
7,2eeac8d8feae4a8cad5a6af0499a211d,3f207df678b143eea3cee63160fa8bed,0,0,1,0
8,aa4862eba776480b8bb9c68455b8c2e1,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0,1,0


In [40]:
# group the offer data, first by customer_id and then by offer_id
temp_offer = offer_data.groupby(['customer_id','offer_id']).agg({'time':'sum', 'offer_completed':'sum', 'offer_received': 'sum', 'offer_viewed': 'sum'})

In [42]:
temp_offer.head(10)

time  \
customer_id                      offer_id                                 
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5  1152   
                                 3f207df678b143eea3cee63160fa8bed   708   
                                 5a8bc65990b245e5a138643cd4eb9837   360   
                                 f19421c1d4aa40978ebb69ca19b0e20d  1278   
                                 fafdcd668e3743c1bb461111dcafc2a4  1572   
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7  1416   
                                 2298d6c36e964ae4a3e7e9706d1fb8c2   606   
                                 3f207df678b143eea3cee63160fa8bed     6   
                                 5a8bc65990b245e5a138643cd4eb9837   690   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9  1596   

                                                                   offer_completed  \
customer_id                      offer_id                                            
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5                1   
                                 3f207df678b143eea3cee63160fa8bed                0   
                                 5a8bc65990b245e5a138643cd4eb9837                0   
                                 f19421c1d4aa40978ebb69ca19b0e20d                1   
                                 fafdcd668e3743c1bb461111dcafc2a4                1   
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7                1   
                                 2298d6c36e964ae4a3e7e9706d1fb8c2                1   
                                 3f207df678b143eea3cee63160fa8bed                0   
                                 5a8bc65990b245e5a138643cd4eb9837                0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9                1   

                                                                   offer_received  \
customer_id                      offer_id                                           
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5               1   
                                 3f207df678b143eea3cee63160fa8bed               1   
                                 5a8bc65990b245e5a138643cd4eb9837               1   
                                 f19421c1d4aa40978ebb69ca19b0e20d               1   
                                 fafdcd668e3743c1bb461111dcafc2a4               1   
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7               1   
                                 2298d6c36e964ae4a3e7e9706d1fb8c2               1   
                                 3f207df678b143eea3cee63160fa8bed               1   
                                 5a8bc65990b245e5a138643cd4eb9837               1   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9               1   

                                                                   offer_viewed  
customer_id                      offer_id                                        
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5             0  
                                 3f207df678b143eea3cee63160fa8bed             1  
                                 5a8bc65990b245e5a138643cd4eb9837             1  
                                 f19421c1d4aa40978ebb69ca19b0e20d             1  
                                 fafdcd668e3743c1bb461111dcafc2a4             1  
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7             1  
                                 2298d6c36e964ae4a3e7e9706d1fb8c2             1  
                                 3f207df678b143eea3cee63160fa8bed             1  
                                 5a8bc65990b245e5a138643cd4eb9837             1  
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9             1

In [46]:
# create customer_offer dataframe
customer_offer = pd.DataFrame(columns=['customer_id', 'offer_id', 'completed'])

for cust in set(offer_data['customer_id']):
    for off in set(offer_data['offer_id']):
        try:
            record = temp_offer.loc[cust,off]
            # in order to complete a offer, a customer should receive, view and then complete an offer
            completed_offer = (record.offer_received == 1 & record.offer_viewed == 1 & record.offer_completed == 1)
            customer_offer = customer_offer.append({'customer_id': cust, 'offer_id': off, 'completed':completed_offer}, ignore_index=True)    
        except:
            continue

In [47]:
customer_offer.head()

,customer_id,offer_id,completed
0,991092556dfe4af39a5e446fd2746ac0,ae264e3637204a6fb9bb56bc8210ddfd,False
1,991092556dfe4af39a5e446fd2746ac0,f19421c1d4aa40978ebb69ca19b0e20d,True
2,991092556dfe4af39a5e446fd2746ac0,4d5c57ea9a6940dd891ad53e9dbe8da0,False
3,991092556dfe4af39a5e446fd2746ac0,fafdcd668e3743c1bb461111dcafc2a4,False
4,a2bfbf0ff27f42bd98fd28be9adb56a1,2906b810c7d4411798c6938adc9daaa5,True


### 5. Create customer_transaction dataframe

In this step my goal is to come up with a new dataframe which contains data related to average spending of a customer. That is, it will contain customer_id, avg_amount_spent

'avg_amount_spent' is calculated as below;
 $
    TotalAmount / TotalNumberOfTransactions
 $
 
Hence, as the first step for it, I need to filter out transaction related rows from transcript dataframe. Afterwards, we can rename 'value' column as 'amount' and drop 'value_type', 'offer completed', 'offer received' and 'offer viewed' columns, since they don't contain any important information for this task.

In [48]:
# Filter out transaction details from transcript dataframe
transaction_data = transcript[transcript['value_type']=='amount']
transaction_data.shape[0]

123957

In [50]:
# do necessary changes to transaction_data 
transaction_data = transaction_data.rename(columns={'value':'amount'})
transaction_data.drop(labels=['value_type','offer_completed','offer_received','offer_viewed'], axis = 1, inplace=True)
transaction_data.head()

,customer_id,amount,time,transaction
12654,02c083884c7d45b39cc68e1314fec56c,0.83,0,1
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,34.56,0,1
12659,54890f68699049c2a04d415abc25e717,13.23,0,1
12670,b2f1cd155b864803ad8334cdf13c4bd2,19.51,0,1
12671,fe97aa22dd3e48c8b143116a8403dd52,18.97,0,1


In [51]:
# creating customer_transaction dataframe
customer_transaction = pd.DataFrame(columns=['customer_id', 'avg_amount_spent'])
temp_trans = transaction_data.groupby('customer_id').agg({'amount':'sum', 'transaction':'sum'})

for cust in set(transaction_data['customer_id']):
    try:
        avg = temp_trans.loc[cust].amount/temp_trans.loc[cust].transaction
        customer_transaction = customer_transaction.append({'customer_id': cust, 'avg_amount_spent': avg}, ignore_index=True)
    except:
        continue
        
customer_transaction.head()

,customer_id,avg_amount_spent
0,a2bfbf0ff27f42bd98fd28be9adb56a1,14.281429
1,991092556dfe4af39a5e446fd2746ac0,4.615000
2,e6ed082e2f7a4f9f94466b7fdb560f05,17.000000
3,7b617719add74bb4a161376cf419c023,13.390000
4,63674022d72c4993a8f6c5edcbf01dcc,3.363750
